### Importing necessary libraries

In [382]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

### Reading data 

In [384]:
df = pd.read_csv('raw_mysuru.csv')

### Performing data cleaning

In [252]:
def formating(x):
    if x.endswith(', Mysore') :
        return x[:-8]
    else:
        return x

In [386]:
df['location'] = df['location'].apply(formating)

In [256]:
# as multiple prokers for same property

### removing duplicates

In [388]:
df.drop_duplicates(subset=['location','price','area','name'],inplace=True)  

### drop unnecessary columns

In [390]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [392]:
df.drop(columns=['name'],inplace=True)

In [570]:
df.isnull().sum()

location          0
price             0
area              0
bhk               0
locations         0
abs_price         0
sqft_val          0
price_per_sqft    0
type              0
dtype: int64

In [572]:
df

,location,price,area,bhk,locations,abs_price,sqft_val,price_per_sqft,type
0,Residential land / Plot in Bogadi,₹17 - 41.25 Lac,"581 - 1,453 sqft",Plot/Land,Bogadi,2912500.0,1017,2863.8,Plot
1,Residential land / Plot in Jayapura,₹12.85 - 43.61 Lac,"680 - 2,560 sqft",Plot/Land,Jayapura,2823000.0,1620,1742.6,Plot
2,Residential land / Plot in Lalithadripura,₹74 Lac,"1,200 sqft",Plot/Land,Lalithadripura,7400000.0,1200,6166.7,Plot
3,3 BHK Flat in Vishweshwara Nagar,₹90 Lac,"1,420 sqft",3 BHK,Vishweshwara Nagar,9000000.0,1420,6338.0,House
4,4 Bedroom House in Vidyaranyapura,₹2.59 Cr,"2,587 sqft",4 BHK,Vidyaranyapura,25900000.0,2587,10011.6,House
...,...,...,...,...,...,...,...,...,...
1745,Residential land / Plot in Bannur Road,₹3 Cr,"10,000 sqft",Plot/Land,Bannur Road,30000000.0,10000,3000.0,Plot
1746,Residential land / Plot in Yelwala,₹25 Lac,800 sqft,Plot/Land,Yelwala,2500000.0,800,3125.0,Plot
1747,3 BHK Flat in Bogadi,₹85 Lac,"1,710 sqft",3 BHK,Bogadi,8500000.0,1710,4970.8,House
1748,6 Bedroom House in Vidyaranyapura,₹1.2 Cr,"1,100 sqft",6 BHK,Vidyaranyapura,12000000.0,1100,10909.1,House


### Finding the hottest locations for real estate in mysore

In [398]:
locations = []
for i in df['location']:
    loc = i.split(' in ')
    locations.append(loc[1])

In [400]:
mysore_areawise = pd.DataFrame(pd.Series(locations).value_counts()).reset_index()

In [402]:
mysore_areawise.columns = ['locations','count']

In [404]:
hot_locations = mysore_areawise.head(30) 

In [406]:
mysore_areawise

,locations,count
0,4th Stage Vijayanagar,71
1,Bogadi,69
2,JP Nagar,61
3,Yelwala,47
4,Hebbal,37
...,...,...
294,Manuganahalli Hunsur road,1
295,Yaraganahalli,1
296,Saligrama,1
297,Vyasthi Serenity Manuganahalli Ilwala Mysore,1


### Applying Feature engineering to include relevant data in required format

In [408]:
def area_extract(x):
    return x.split(' in ')[1]

In [410]:
df['locations'] = df['location'].apply(area_extract)

In [412]:
df['price'].apply(lambda x : 'Lac' if x.endswith('Lac') else ('Cr' if x.endswith('Cr') else x)).value_counts()

price
Lac                 907
Cr                  588
Price on Request      1
Name: count, dtype: int64

In [498]:
def extract_float(s):
    s_clean = s.replace(',', '')
    
    pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
    match = pattern.search(s_clean)
    
    return float(match.group()) if match else None

print(extract_float('₹49,99 Lac'))

4999.0


In [454]:
def new_price(price):
    price_new = []
    for i in price:
        if ' - ' in i:
            # via manual screening we know suffix = Lac
            suffix = 100000
            vals = i.split(' - ')
            num1 = extract_float(vals[0])
            num2 = extract_float(vals[1])
            price_new.append(round(((num1+num2)/2)*suffix))
        else:
            if i.endswith('Lac'):
                suffix = 100000
            elif i.endswith('Cr'):
                suffix = 10000000
            val = extract_float(i)
            try:
                price_new.append(round(val*suffix))
            except Exception as e:
                price_new.append(np.nan)
                print(e)
    #print(price_new)
    return price_new

In [488]:
pd.Series(new_price(df['price'])).isnull().sum()

unsupported operand type(s) for *: 'NoneType' and 'int'


1

In [490]:
df['abs_price'] = pd.Series(new_price(df['price']),index=df.index)

unsupported operand type(s) for *: 'NoneType' and 'int'


In [492]:
df['abs_price'].isnull().sum()

1

In [516]:
def extract_sqft(area):
    sqft = []
    for i in area:
        if ' - ' in i:
            vals = i.split(' - ')
            num1 = extract_float(vals[0])
            num2 = extract_float(vals[1])
            sqft.append(round((num1+num2)/2))
        else:
            val = extract_float(i)
            try:
                sqft.append(round(val))
            except Exception as e:
                sqft.append(np.nan)
                print(e)
    return sqft

In [522]:
df['sqft_val'] = pd.Series(extract_sqft(df['area']),index=df.index)

In [544]:
df['price_per_sqft'] = round(df['abs_price']/df['sqft_val'],1)

In [552]:
df.dropna(inplace=True)

In [568]:
df['type'] = df['bhk'].apply(lambda x : 'Plot' if x == 'Plot/Land' else 'House')

In [583]:
df['num_bhk'] = df['bhk'].apply(extract_float)

In [606]:
avg_price_data = pd.DataFrame(df.groupby(['locations', 'type'])['price_per_sqft'].mean()).reset_index()

In [610]:
avg_price_data['price_per_sqft'] = avg_price_data['price_per_sqft'].apply(lambda x : round(x))

### Exporting cleaned and transformed data 

In [638]:
avg_price_data.to_csv('avg.csv')

df.to_csv('mysuru_fin.csv')